In [1]:
import ROOT
import edm4hep
from podio import root_io
from math import sqrt

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x80561b0
Welcome to JupyROOT 6.28/10


In [2]:
%jsroot on

In [3]:
ROOT.gInterpreter.Declare("#include <marlinutil/HelixClass_double.h>")
# ROOT.gInterpreter.Declare("#include <marlinutil/GeometryUtil.h>")

True

In [4]:
# Meh would require dd4hep
# Bz = ROOT.MarlinUtil.getBzAtOrigin()
Bz = 2
print(Bz)

2


In [5]:
# reader = root_io.Reader("E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.10.REC_edm4hep.root")
reader = root_io.Reader("E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.1000.REC_edm4hep.root")

In [6]:
events = reader.get("events")


In [7]:
h_omega = ROOT.TH1D("h_omega", ";pull #Omega", 50, -10, 10)
h_phi = ROOT.TH1D("h_phi", ";pull #phi", 50, -10, 10)
h_momentum = ROOT.TH1D("h_momentum", "; E [GeV]", 250, 0, 125)
h_omega_2d = ROOT.TH2D("h_omega_2d", "; E [GeV]; pull #Omega", 10, 0, 125, 50, -10, 10)

In [8]:
for event in events:
    # mctruthrecolink = event.get("MCTruthSiTracksLink")
    mctruthrecolink = event.get("SiTracksMCTruthLink")
    for link in mctruthrecolink:
        mcp = link.getSim()
        reco = link.getRec()
        weight = link.getWeight()
        # require at least a bit of quality
        # FIXME: ah lol does this direction in the relation kill electrons with high brems loss as the fraction of their initial energy going to the track is lower?
        if (weight < 0.9):
            continue
        # only care about electrons
        if (abs(mcp.getPDG()) != 11 or mcp.getGeneratorStatus() != 1):
            continue
        # require at least some energy
        # FIXME: probably should cut on pT instead
        if (mcp.getEnergy() < 1):
            continue

        reco_track = reco.getTrackStates(0)
        cov = reco_track.covMatrix
        mc_track = ROOT.HelixClass_double()
        mc_pos = mcp.getVertex()
        mc_pos_v = ROOT.std.vector["double"]((mc_pos.x, mc_pos.y, mc_pos.z))
        mc_mom = mcp.getMomentum()
        mc_mom_v = ROOT.std.vector["double"]((mc_mom.x, mc_mom.y, mc_mom.z))
        mc_track.Initialize_VP(mc_pos_v.data(), mc_mom_v.data(), mcp.getCharge(), 2)
        pull_omega = (reco_track.omega - mc_track.getOmega()) / sqrt(cov[5])
        pull_phi = (reco_track.phi - mc_track.getPhi0()) / sqrt(cov[2])
        # print(f"energy: {mcp.getEnergy()}")
        # print(f"reco omega: {reco_track.omega}")
        # print(f"mc omega {mc_track.getOmega()}")
        # print(f"sig_omega: {sqrt(cov[5])}")
        # print(f"pull omega: {pull_omega}")
        h_omega.Fill(pull_omega)
        h_phi.Fill(pull_phi)
        h_momentum.Fill(mcp.getEnergy())
        h_omega_2d.Fill(mcp.getEnergy(), pull_omega)



In [9]:
c_omega = ROOT.TCanvas()
h_omega.Draw()
h_omega.Fit("gaus")
c_omega.Draw()

c_phi = ROOT.TCanvas()
h_phi.Draw()
h_phi.Fit("gaus")
c_phi.Draw()

c_e = ROOT.TCanvas()
h_momentum.Draw()
c_e.Draw()

 FCN=155.651 FROM MIGRAD    STATUS=CONVERGED      70 CALLS          71 TOTAL
                     EDM=9.38732e-11    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.4 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.44610e+01   3.01780e+00   4.26099e-03   4.03656e-06
   2  Mean        -1.34719e-01   8.94974e-02  -3.11949e-05   3.73054e-05
   3  Sigma        2.12244e+00   1.14611e-01  -2.10930e-05  -4.28960e-04
 FCN=70.0217 FROM MIGRAD    STATUS=CONVERGED      65 CALLS          66 TOTAL
                     EDM=1.38017e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.37160e+01   1.33011e+00   4.56034e-03   2.35025e-04
   2  Mean        -6.98662e-02   1.67240e-01   6.85213e-04   9.02701e-04
   3  Sigma        3

In [10]:
c_2d = ROOT.TCanvas()
h_omega_2d.Draw("colz")
c_2d.Draw()